In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm


In [ ]:
S_0=100
K=100
r=0.05
sigma=0.50
T=1
N=252
dt=T/N
n_sims=10**6
discount_factor=np.exp(-r*T)

In [ ]:
def black_scholes(S_0,K,T,r,sigma,type='call'):
    d1=(np.log(S_0/K)+(r+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    d2=(np.log(S_0/K)+(r-0.5*sigma**2)*T)/(sigma*np.sqrt(T))

    if type=='call':
        val=(S_0*norm.cdf(d1,0,1)-K*np.exp(-r*T)*norm.cdf(d2,0,1))

    elif type=='put':
        val=(K*np.exp(-r*T)*norm.cdf(-d2,0,1)-S_0*norm.cdf(-d1,0,1))


    return val

In [ ]:
black_scholes(S_0=S_0,K=K,T=T,r=r,sigma=sigma,type='call')

21.79260421286685

In [ ]:
black_scholes(S_0=S_0, K=K, T=T, r=r, sigma=sigma, type='put')

16.915546662938254

In [ ]:
def simulate_gbm(s_0, mu, sigma, n_sims, T, N, random_seed=42, antithetic_var=False):
    '''
    Function used for simulating stock returns using Geometric Brownian Motion.
    
    Parameters
    ----------
    s_0 : float
        Initial stock price
    mu : float
        Drift coefficient
    sigma : float
        Diffusion coefficient
    n_sims : int
        Number of simulations paths
    dt : float
        Time increment, most commonly a day
    T : float
        Length of the forecast horizon, same unit as dt
    N : int
        Number of time increments in the forecast horizon
    random_seed : int
        Random seed for reproducibility
    antithetic_var : bool
        Boolean whether to use antithetic variates approach to reduce variance
    Returns
    -------
    S_t : np.ndarray
        Matrix (size: n_sims x (T+1)) containing the simulation results. 
        Rows respresent sample paths, while columns point of time.
    '''

    np.random.seed(random_seed)

    # time increment
    dt = T/N

    # Brownian
    if antithetic_var:
        dW_ant = np.random.normal(scale=np.sqrt(dt),
                                  size=(int(n_sims/2), N + 1))
        dW = np.concatenate((dW_ant, -dW_ant), axis=0)
    else:
        dW = np.random.normal(scale=np.sqrt(dt),
                              size=(n_sims, N + 1))

    # simulate the evolution of the process
    S_t = s_0 * np.exp(np.cumsum((mu - 0.5 * sigma ** 2) * dt + sigma * dW,
                                 axis=1))
    S_t[:, 0] = s_0

    return S_t


In [ ]:
gbm_sims=simulate_gbm(s_0=S_0,mu=r,sigma=sigma,n_sims=n_sims,T=T,N=N)

In [ ]:
premieum=discount_factor*np.average(np.maximum(0,gbm_sims[:,-1]-K))
premieum

21.756178586245806